In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sneaker-sales/2019 Winter Data Science Intern Challenge Data Set - Sheet1.csv


# Question 1:
    On Shopify, we have exactly 100 sneaker shops, and each of these shops sells only one model of shoe. We want to do some analysis of the average order value (AOV). When we look at orders data over a 30 day window, we naively calculate an AOV of $3145.13. Given that we know these shops are selling sneakers, a relatively affordable item, something seems wrong with our analysis.

## Short answers:

### a) Think about what could be going wrong with our calculation. Think about a better way to evaluate this data. 
    Firstly, the dataset may contain duplicates and outliers that affect the further analysis. Not removing them might have impact on the calcualtion of AOV. After some initial screening of duplicates and outliers, the naively calculated AOV of $3145.13 is apparently affected by some extreme order_amount values. It's better to use adjusted dataset after removing duplicates and outliers to calculate the adjusted AOV.
    
### b) What metric would you report for this dataset?
     Depends on the further investigations about the potential fraudualent behaviors, there are two potential metrics that we could use to describe this dataset:
    1. If the high price tag of shop 78 is legit, then records of shop 78 should not be removed and the dataset is highly skewed. With that, a median order value （MOV） would be a better metric to measure the central tendency of the dataset;
    2. If the records of shop 78 should be removed, then using a adjusted average order value (ADAOV) after removing duplicates and outliers from the original dataset is appropraite.
 
### c) What is its value?
    1. Median order value （MOV）$284.0 
    2. Adjusted average order values (ADAOV) $302.58051448247926


    Please see the detailed programming below for more explainations.

## Setting up

In [2]:
import warnings
warnings.filterwarnings(action = 'ignore')

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

## Import data

In [3]:
df = pd.read_csv('../input/sneaker-sales/2019 Winter Data Science Intern Challenge Data Set - Sheet1.csv')

## General data information

In [4]:
df.head()

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
0,1,53,746,224,2,cash,2017-03-13 12:36:56
1,2,92,925,90,1,cash,2017-03-03 17:38:52
2,3,44,861,144,1,cash,2017-03-14 4:23:56
3,4,18,935,156,1,credit_card,2017-03-26 12:43:37
4,5,18,883,156,1,credit_card,2017-03-01 4:35:11


In [5]:
df.describe()

,order_id,shop_id,user_id,order_amount,total_items
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000
mean,2500.500000,50.078800,849.092400,3145.128000,8.78720
std,1443.520003,29.006118,87.798982,41282.539349,116.32032
min,1.000000,1.000000,607.000000,90.000000,1.00000
25%,1250.750000,24.000000,775.000000,163.000000,1.00000
50%,2500.500000,50.000000,849.000000,284.000000,2.00000
75%,3750.250000,75.000000,925.000000,390.000000,3.00000
max,5000.000000,100.000000,999.000000,704000.000000,2000.00000


**Observations:**

* Order_amount has a mean of 3145.128000 and standard deviation (std) of 41282.539349 suggesting a skewed distribution of the order_amount data.
* Same as order_amount, total_items also shows a skewed distribution with a mean of 8.78720 and a std of 116.32032.
* Both evidence indicate that the dataset might contain **outliers with significant impact**.

In [6]:
df.nunique()

order_id          5000
shop_id            100
user_id            301
order_amount       258
total_items          8
payment_method       3
created_at        4991
dtype: int64

## Checking missing values

In [7]:
df.isna().sum()

order_id          0
shop_id           0
user_id           0
order_amount      0
total_items       0
payment_method    0
created_at        0
dtype: int64

**Observations:**
* There is **no missing value** in our data set. We are all set to perform further exploratory data analysis.

## Checking duplicates

* Here, we defined duplicates as orders that are having exact everything except order_id. The reasoning behind this is that for legit purchases, it's nearly impossible to order same goods at the same time in same shop.

In [8]:
duplicated = df[df.duplicated(subset = ['shop_id','user_id','order_amount','total_items','payment_method','created_at'], keep = False)]
duplicated.head(100)

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
15,16,42,607,704000,2000,credit_card,2017-03-07 4:00:00
520,521,42,607,704000,2000,credit_card,2017-03-02 4:00:00
1104,1105,42,607,704000,2000,credit_card,2017-03-24 4:00:00
2297,2298,42,607,704000,2000,credit_card,2017-03-07 4:00:00
2835,2836,42,607,704000,2000,credit_card,2017-03-28 4:00:00
2969,2970,42,607,704000,2000,credit_card,2017-03-28 4:00:00
3332,3333,42,607,704000,2000,credit_card,2017-03-24 4:00:00
4056,4057,42,607,704000,2000,credit_card,2017-03-28 4:00:00
4646,4647,42,607,704000,2000,credit_card,2017-03-02 4:00:00


* Obviously, there are some duplicates came from **user_id 607** and **shop_id 42** as orders were placed at exact same times. Despite the fact that those transactions looks suspicious, let's remove the duplicates and do further outliers analysis in the following section.

In [9]:
df = df.drop_duplicates(subset = ['shop_id','user_id','order_amount','total_items','payment_method','created_at'])
df.nunique()

order_id          4995
shop_id            100
user_id            301
order_amount       258
total_items          8
payment_method       3
created_at        4991
dtype: int64

* Looks like we sussecfully **removed 5 duplicated** orders.

## Exploratory data analysis (EDA) and initial screening on outliers

* As mentioned in the last section, the distributions of order_amount and total_items seem **skewed** and **contain outliers**. Let's visualize the distributions. 

In [10]:
def plot_histogram(feature,central_tendency,insert_line):
    fig = px.histogram(df, x = feature,
                       marginal = "violin",
                       color_discrete_sequence = ['salmon','lightblue'],
                       hover_data = df.columns,
                       log_y = True,
                       log_x = False,
                       nbins = 100
    )
    fig.update_layout(
        title = {
            'text': feature +' histogram',
            'x':0.5,
            'xanchor':'center',
            'yanchor':'top',
        },
        height = 600,
        width = 1000,
        paper_bgcolor = 'White',
        plot_bgcolor = 'GhostWhite',
    )
    fig.add_annotation(
        x = insert_line,
        text= central_tendency + ' = '+ str(insert_line),
        font=dict(
            size=16,
            ),
        showarrow=True,
        arrowhead=3,
        arrowsize=1,
        arrowwidth=2,
        yshift = 15,
        ax=100,
        ay=-100,
    )
    fig.add_shape(
        type='line',
        x0=insert_line, y0=0.1, 
        x1=insert_line, y1=5000,
        line=dict(color='RoyalBlue',width=3),
    )
    fig.show()

In [11]:
plot_histogram('order_amount','AOV',df['order_amount'].mean())

In [12]:
plot_histogram('total_items','Average items/order',df['total_items'].mean())

**Observations:**
* There are some extreme values exist in both order_amount and total_items

* Let's take a look at the characteristics and patterns of those extream values. Here I define orders that have order_amount > 200k or total_items > 10 as extream values.


In [13]:
order_amount_extream = df[df['order_amount']>200000]
order_amount_extream.nunique()

order_id          12
shop_id            1
user_id            1
order_amount       1
total_items        1
payment_method     1
created_at        12
dtype: int64

In [14]:
total_items_extream = df[df['total_items']>10]
total_items_extream.nunique()

order_id          12
shop_id            1
user_id            1
order_amount       1
total_items        1
payment_method     1
created_at        12
dtype: int64

In [15]:
total_items_extream.head()

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
15,16,42,607,704000,2000,credit_card,2017-03-07 4:00:00
60,61,42,607,704000,2000,credit_card,2017-03-04 4:00:00
520,521,42,607,704000,2000,credit_card,2017-03-02 4:00:00
1104,1105,42,607,704000,2000,credit_card,2017-03-24 4:00:00
1362,1363,42,607,704000,2000,credit_card,2017-03-15 4:00:00


* Apparently the outliers are associated to a **single shop 42** and/or a **single user 607**. So is outlier the shop or the user?
Let's find it out.

## Futher ourlier detection

In [16]:
df_shop42 = df[df['shop_id'] ==42]
df_shop42.head(10)

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
15,16,42,607,704000,2000,credit_card,2017-03-07 4:00:00
40,41,42,793,352,1,credit_card,2017-03-24 14:15:41
60,61,42,607,704000,2000,credit_card,2017-03-04 4:00:00
308,309,42,770,352,1,credit_card,2017-03-11 18:14:39
409,410,42,904,704,2,credit_card,2017-03-04 14:32:58
520,521,42,607,704000,2000,credit_card,2017-03-02 4:00:00
834,835,42,792,352,1,cash,2017-03-25 21:31:25
835,836,42,819,704,2,cash,2017-03-09 14:15:15
938,939,42,808,1056,3,credit_card,2017-03-13 23:43:45
979,980,42,744,352,1,debit,2017-03-12 13:09:04


* From the above table we can tell that shop 42 has multiple customers and most of the customers conducts normal purchasing behaviors(small item numbers and low order_amount). So let's take a look at user 607.

In [17]:
df_user607 = df[df['user_id'] == 607]
df_user607.head(100)

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
15,16,42,607,704000,2000,credit_card,2017-03-07 4:00:00
60,61,42,607,704000,2000,credit_card,2017-03-04 4:00:00
520,521,42,607,704000,2000,credit_card,2017-03-02 4:00:00
1104,1105,42,607,704000,2000,credit_card,2017-03-24 4:00:00
1362,1363,42,607,704000,2000,credit_card,2017-03-15 4:00:00
1436,1437,42,607,704000,2000,credit_card,2017-03-11 4:00:00
1562,1563,42,607,704000,2000,credit_card,2017-03-19 4:00:00
1602,1603,42,607,704000,2000,credit_card,2017-03-17 4:00:00
2153,2154,42,607,704000,2000,credit_card,2017-03-12 4:00:00
2835,2836,42,607,704000,2000,credit_card,2017-03-28 4:00:00


* Therefore, user 607 has very fraudulent behavior as the purchases are all 704000 in order_amount, 2000 in total_items, shopped in shop 42 at 4 o'clock with credit card. Let's remove user 607 and recheck the distribution of total_items.

In [18]:
df = df[df['user_id'] != 607]
df.nunique()

order_id          4983
shop_id            100
user_id            300
order_amount       257
total_items          7
payment_method       3
created_at        4979
dtype: int64

In [19]:
items_per_shop = df.groupby('shop_id').sum()[['total_items']].sort_values('total_items',ascending = False)
items_per_shop = items_per_shop.reset_index()
items_per_shop

,shop_id,total_items
0,13,136
1,84,132
2,71,130
3,53,130
4,81,128
...,...,...
95,44,71
96,16,71
97,56,69
98,85,67


In [20]:
def plot_bar(feature,yaxis,comment):
    fig = px.bar(feature, 
             x = 'shop_id', y = yaxis,
             color_discrete_sequence = ['salmon','lightblue'],
        )
    fig.update_layout(
        title = {
            'text': yaxis +' barchart '+ comment,
            'x':0.5,
            'xanchor':'center',
            'yanchor':'top',
        },
        height = 600,
        width = 1000,
        paper_bgcolor = 'White',
        plot_bgcolor = 'GhostWhite',
        )
    fig.show()

In [21]:
plot_bar(items_per_shop,'total_items','w/o user 607')

* The total_items of shops seem evenly distributed after removing records of user 607. Now is the time to exam the order_amount.

In [22]:
order_per_shop = df.groupby('shop_id').sum()[['order_amount']].sort_values('order_amount',ascending = False)
order_per_shop.head()

,order_amount
shop_id,
78,2263800
89,23128
81,22656
6,22627
42,22176


* Remember we have a **mean order_amount** of **$3145.128000** and a std of **$41282.539349**. The order_amounts of shop **78** is significantly higher than next selling shop in the list which leads to possibilities:

1. Shop 78 sells way more sneakers than other shops;
2. Shop 78 sells sneakers in very high prices.

* As shown in the last figure, there is no shop has outstanding amount of item sold. So is the retail price of shop 78 too high? Here, we calculate the **average price per item** sold by each shop and compare them with each other.

In [23]:
avg_price = df.groupby('shop_id').apply(lambda x:x['order_amount']/x['total_items'])
avg_price.groupby('shop_id').first().sort_values(ascending=False).head(5)

shop_id
78    25725.0
42      352.0
12      201.0
89      196.0
99      195.0
dtype: float64

* Shop **78** has the highest average price per item of all shops. **$25725.0 per pair of shoes** seems suspiciously high for sneakers. Does shop 78 even sell sneakers with normal price tag?

In [24]:
df_shop78 = df[df['shop_id'] == 78]
df_shop78['avg_price'] = df_shop78['order_amount']/df_shop78['total_items']
df_shop78['avg_price']

160     25725.0
490     25725.0
493     25725.0
511     25725.0
617     25725.0
691     25725.0
1056    25725.0
1193    25725.0
1204    25725.0
1259    25725.0
1384    25725.0
1419    25725.0
1452    25725.0
1529    25725.0
2270    25725.0
2452    25725.0
2492    25725.0
2495    25725.0
2512    25725.0
2548    25725.0
2564    25725.0
2690    25725.0
2773    25725.0
2818    25725.0
2821    25725.0
2906    25725.0
2922    25725.0
3085    25725.0
3101    25725.0
3151    25725.0
3167    25725.0
3403    25725.0
3440    25725.0
3705    25725.0
3724    25725.0
3780    25725.0
4040    25725.0
4079    25725.0
4192    25725.0
4311    25725.0
4412    25725.0
4420    25725.0
4505    25725.0
4584    25725.0
4715    25725.0
4918    25725.0
Name: avg_price, dtype: float64

* The answer is no. Shop 78 only sales sneaker with a $25725 price tag. Let's remove shop 78 from the list and recheck the distribution of order_amount.

In [25]:
df = df[df['shop_id']!=78]
order_per_shop = df.groupby('shop_id').sum()[['order_amount']].sort_values('order_amount',ascending = False)
order_per_shop = order_per_shop.reset_index()
order_per_shop

,shop_id,order_amount
0,89,23128
1,81,22656
2,6,22627
3,42,22176
4,13,21760
...,...,...
94,2,9588
95,100,8547
96,56,8073
97,32,7979


In [26]:
plot_bar(order_per_shop,'order_amount','w/o shop 78')

* After removing shop 78, there is no outstanding order_amount in the dataset. **Now it's time to calculate the adjusted average order value (ADAOV) after removing outlier user 607 and shop 48**. 

## Calculate the adjusted average order values (ADAOV)

In [27]:
df.describe()

,order_id,shop_id,user_id,order_amount,total_items
count,4937.000000,4937.000000,4937.000000,4937.000000,4937.000000
mean,2499.551347,49.846465,849.752279,302.580514,1.994734
std,1444.069407,29.061131,86.840313,160.804912,0.982821
min,1.000000,1.000000,700.000000,90.000000,1.000000
25%,1248.000000,24.000000,775.000000,163.000000,1.000000
50%,2497.000000,50.000000,850.000000,284.000000,2.000000
75%,3751.000000,74.000000,925.000000,387.000000,3.000000
max,5000.000000,100.000000,999.000000,1760.000000,8.000000


* Now our adjusted AOV is 302.58 with a much reasonable std of 160.80. Let's compare it with another common measure of central tendency - median.

In [28]:
mean = df['order_amount'].mean()
median = df['order_amount'].median()

print(f'The adjusted average order values (ADAOV) after removing outliers is ${mean}')
print(f'The median order values (MOV) is ${median}')

The adjusted average order values (ADAOV) after removing outliers is $302.58051448247926
The median order values (MOV) is $284.0


# Question 2:

### a) How many orders were shipped by Speedy Express in total?

SELECT s.ShipperID, s.ShipperName, COUNT(*) AS OrderCount
FROM Orders o
INNER JOIN Shippers s
ON o.ShipperID = s.ShipperID
WHERE s.ShipperName = 'Speedy Express'

The total order shipped by Speedy Express is **54**.

### b) What is the last name of the employee with the most orders?

SELECT e.EmployeeID, e.LastName, COUNT(*) AS OrderCount
FROM Orders o
INNER JOIN Employees e
ON o.EmployeeID = e.EmployeeID
GROUP BY e.EmployeeID
ORDER BY OrderCount DESC

It appers that the last name of the employee with the most orders is **Peacock** who parocessed **40 orders**.

### c) What product was ordered the most by customers in Germany?

SELECT c.Country, p.ProductName, SUM(od.Quantity) AS MostOrdered
FROM OrderDetails od
JOIN Products p
JOIN Customers c
JOIN Orders o
ON od.OrderID = o.OrderID AND
    od.ProductID = p.ProductID AND
    o.CustomerID = c.CustomerID
WHERE c.Country = 'Germany'
GROUP BY od.ProductID
ORDER BY MostOrdered DESC

**Boston Crab Meat**, which has been ordered **160 times**, seems to be the most ordered food by **Germany**.